# Example 2: Extract method

[Extract method in the refactoring catalog.](http://refactoring.com/catalog/extractMethod.html)

---

Do this when a function is getting too long, doing too many things, or needs to maintain state through loop iterations.

In [15]:
class Grade:
    def __init__(self, student, score):
        self.student = student
        self.score =  score


def print_stats(grades):
    if not grades:
        raise ValueError('Must supply at least one Grade')
        
    total, count = 0, 0
    low, high = float('inf'), float('-inf')
    for grade in grades:
        total += grade.score
        count += 1
        if grade.score < low:
            low = grade.score
        elif grade.score > high:
            high = grade.score

    average = total / count

    print('Average score: %.1f, low score: %.1f, high score %.1f' %
          (average, low, high))

In [16]:
grades = [Grade('Bob', 92), Grade('Sally', 89), Grade('Roger', 73), Grade('Alice', 96)]
print_stats(grades)

Average score: 87.5, low score: 73.0, high score 96.0


---

One bad way to try to do this is with a closure. But this is ugly because you need to use the `nonlocal` keyword. In Python 2 it's even worse.

In [35]:
def print_stats(grades):
    if not grades:
        raise ValueError('Must supply at least one Grade')
        
    total, count = 0, 0
    low, high = float('inf'), float('-inf')

    def adjust_stats(grade):
        nonlocal total, count, low, high
        total += grade.score
        count += 1
        if grade.score < low:
            low = grade.score
        elif grade.score > high:
            high = grade.score

    for grade in grades:
        adjust_stats(grade)
            
    average = total / count

    print('Average score: %.1f, low score: %.1f, high score %.1f' %
          (average, low, high))

In [36]:
print_stats(grades)

Average score: 87.5, low score: 73.0, high score 96.0


---

What's better is to split the inner closure into a helper class. You make the helper class having a single entrypoint named `__call__` so it acts like a plain function. This is a hint to the reader than the purpose of the class is to be a stateful closure.

In [38]:
class CalculateStats:
    def __init__(self):
        self.total = 0
        self.count = 0
        self.low = float('inf')
        self.high = float('-inf')

    def __call__(self, grades):
        for grade in grades:
            self.total += grade.score
            self.count += 1
            if grade.score < self.low:
                self.low = grade.score
            elif grade.score > self.high:
                self.high = grade.score

                
def print_stats(grades):
    if not grades:
        raise ValueError('Must supply at least one Grade')

    stats = CalculateStats()
    stats(grades)
    average = stats.total / stats.count

    print('Average score: %.1f, low score: %.1f, high score %.1f' %
          (average, stats.low, stats.high))

In [39]:
print_stats(grades)

Average score: 87.5, low score: 73.0, high score 96.0


---

You can even add other properties to this closure to give the illusion it's doing more bookkeeping than it really is.

In [40]:
class CalculateStats:
    def __init__(self):
        self.total = 0
        self.count = 0
        self.low = float('inf')
        self.high = float('-inf')

    def __call__(self, grades):
        for grade in grades:
            self.total += grade.score
            self.count += 1
            if grade.score < self.low:
                self.low = grade.score
            elif grade.score > self.high:
                self.high = grade.score

    @property
    def average(self):
        return self.total / self.count

    
def print_stats(grades):
    if not grades:
        raise ValueError('Must supply at least one Grade')

    stats = CalculateStats()
    stats(grades)

    print('Average score: %.1f, low score: %.1f, high score %.1f' %
          (stats.average, stats.low, stats.high))

In [41]:
print_stats(grades)

Average score: 87.5, low score: 73.0, high score 96.0


---

If you need more than one entrypoint method, you probably need to redraw the boundaries of responsibility between the classes and go for real method names, not just `__call__`.